# 磁力線追跡

##### モジュールのインポートとRunge-Kuttaに渡す関数の定義

In [13]:
import sub.plot as pl
import sub.functions as sb
import sub.electromagnetics as sel
from global_variables import gparam
import numpy as np
import vessel.vmat as svm
import sub.runge_kutta as ruke

gl = gparam()

In [14]:
# Runge-Kuttaに渡す磁場の関数の定義
class Rmag(sel.Magnetic):
    direction = 1
    def get_val(self, t0, x0):
        b = np.array(self.get_mag(x0))
        b0 = b/np.sqrt(np.sum(b*b)) # normalization
        return b0*self.direction
    def is_end(self, t0, x0):
        x, y, z = x0
        r = np.sqrt(x**2+y**2)
        return not svm.is_inside_vac(r, z)

##### 磁場の計算条件の定義

In [18]:
condition = {
    # TF current
    'cur_tf':{'tf': +50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':+0.0e+3, 'r0':0.65, 'z0':0.0, 'radius':0.3},
    
    # PF currents
    'cur_pf':{'pf17t12':-1.0e+3, 'pf26t36':-1.0e+3,'pf4_ab_cc2':-5.0e+3, },
    
    'resolution': gl.get_dmat_coarse(),
    }

##### 磁場の計算と計算結果の確認

In [19]:
# 磁場の計算
cond = sb.equi_pre_process(condition, verbose=0)

In [20]:
# 計算結果の確認
pl.d_heatmap(cond['vessel']) # 真空容器
pl.d_contour(cond['flux']) # flux分布

##### 磁力線の座標を返す関数の定義

In [ ]:
# Runge-Kuttaに渡す関数の作成
mag = Rmag(cond['flux'], cond['pol_current'])

In [ ]:
# 磁力線の座標を返す関数の定義
def calc_mag_trace(initial_point, step_width=0.01, step_num=10000):
    """磁力線の座標を返す。

    Args:
        initial_point (array): [x, y, z]
        step_width (float): ステップ幅
        step_num (int): イタレーション回数
    """
    p0 = np.array(initial_point)
    crk = ruke.Runge_kutta(mag)
    mag.direction = 1
    pp = crk.solve(0, p0, step_width, step_num)
    mag.direction = -1
    pn = crk.solve(0, p0, step_width, step_num)
    pts = np.vstack((pn[1:][::-1], pp)) # pn[1:][::-1] 重複する最初のデータを削除して反転
    return pts

##### 磁力線の座標の計算と計算結果の確認

In [ ]:
# 磁力線の座標の計算
i_point = [0.5, 0.0, 0.0]
s_width=0.01
pts = calc_mag_trace(i_point, s_width)
print(f'磁力線長さ：{len(pts)*s_width} m')

In [ ]:
pl.line_plot3d(pts) # 3D表示
pl.line_plot(pts) # ポロイダル断面への投影